# Detect all faces, extract facial features, save sub-images of faces.

In [2]:
%pylab inline

import os
import PIL
from PIL import Image
from torch.autograd import Variable
from glob import glob
import pickle
from tqdm import tqdm

def readAndSplit(p):
    I = imread(p)
    I = I[:245,:400,:3]
    s = hsplit(I,2)
    return s[0],s[1]

Populating the interactive namespace from numpy and matplotlib


In [3]:
import face_recognition
baseDir = os.path.join(os.path.expanduser('~/TotallyLooksLike'))
imageDir=os.path.join(baseDir,'Images')
outputDir=os.path.join(baseDir,'results')
# just the faces.
imageNames = []
G = sorted(glob(os.path.join(imageDir,'*.jpg')))
faces_path = os.path.join(outputDir,'faces.pkl')

In [4]:
import face_recognition
baseDir = os.path.join(os.path.expanduser('~/TotallyLooksLike'))
imageDir=os.path.join(baseDir,'Images')
outputDir=os.path.join(baseDir,'results')
# just the faces.
imageNames = []
G = sorted(glob(os.path.join(imageDir,'*.jpg')))
faces_path = os.path.join(outputDir,'faces.pkl')
if os.path.isfile(faces_path):
    imgs_to_faces = pickle.load(open(faces_path,'rb'))
else:
    imgs_to_faces={}
    for imgPath in tqdm(G):
        image = face_recognition.load_image_file(imgPath)
        face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="cnn")
        # face_locations is a list of : [top, right, bottom,left]
        # to crop: face_image = image[top:bottom, left:right]
        imgs_to_faces[imgPath.split('/')[-1]] = face_locations
    
    pickle.dump(imgs_to_faces,open(faces_path,'wb'))

In [5]:
# faces, locations, encodings, face images.
imageNames = []
import PIL
from PIL import Image
from torch.autograd import Variable
import os
from glob import glob
import pickle
from tqdm import tqdm

G = sorted(glob(os.path.join(imageDir,'*.jpg')))
faces_enc_path = os.path.join(outputDir,'face_encodings.pkl')
if os.path.isfile(faces_enc_path):
    imgs_to_face_enc = pickle.load(open(faces_enc_path,'rb'))

else:
    imgs_to_face_enc={}
    #for ii,imgPath in enumerate(G):#tqdm(G):
    for imgPath in tqdm(G):        
        #image = face_recognition.load_image_file(imgPath)
        
        #face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="cnn")
        # face_locations is a list of : [top, right, bottom,left]
        # to crop: face_image = image[top:bottom, left:right]
        #I = imread(imgPath)
        #print(I.shape)
        s = readAndSplit(imgPath)
        leftImage = s[0]
        rightImage = s[1]
        break
        face_locations_left = face_recognition.face_locations(leftImage, number_of_times_to_upsample=0, model="cnn")
        face_locations_right = face_recognition.face_locations(rightImage, number_of_times_to_upsample=0, model="cnn")
        left_face_enc = face_recognition.face_encodings(leftImage,known_face_locations=face_locations_left)
        right_face_enc = face_recognition.face_encodings(rightImage,known_face_locations=face_locations_right)
        
        # also keep all the face images.
        left_face_imgs = []
        right_face_imgs = []        
        
        for top, right, bottom, left in face_locations_left:
            left_face_imgs.append(leftImage[top:bottom, left:right])
        for top, right, bottom, left in face_locations_right:            
            right_face_imgs.append(rightImage[top:bottom, left:right])
                
        imgs_to_face_enc[imgPath.split('/')[-1]] = dict(left_enc=left_face_enc,
                                                        right_enc=right_face_enc,
                                                        left_loc=face_locations_left,
                                                        right_loc=face_locations_right,
                                                        left_face_imgs=left_face_imgs,
                                                        right_face_imgs=right_face_imgs)
                                      
    pickle.dump(imgs_to_face_enc,open(faces_enc_path,'wb'),protocol=2); # for compatability with python 2.7